# 세티즌 

## 세티즌 크롤링

In [1]:
#!pip install webdriver_manager

In [2]:
#!pip install selenium

In [5]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [6]:
from tqdm import tqdm

In [14]:
url='https://market.cetizen.com/market.php?q=market&m%5B12%5D=12&tabname=pills-brand&p=1'
req=requests.get(url,'html.parser')

In [15]:
soup=BeautifulSoup(req.text)

In [61]:
temp=soup.select('#condition_Form > div.row.border-left.border-right > div.col-9.pt-3 > div.row.pl-3.py-3 > div > div > div.col-10.px-3 > div.row.text-left.ml-2 > div.col-10.text-truncate.py-0.my-0 > a')

In [65]:
link_list=[] ## 개별 링크 저장
for page in tqdm(range(1,94)):
    url='https://market.cetizen.com/market.php?q=market&m%5B12%5D=12&tabname=pills-brand&p='+str(page)
    req=requests.get(url,'html.parser')
    soup=BeautifulSoup(req.text)
    temp=soup.select('#condition_Form > div.row.border-left.border-right > div.col-9.pt-3 > div.row.pl-3.py-3 > div > div > div.col-10.px-3 > div.row.text-left.ml-2 > div.col-10.text-truncate.py-0.my-0 > a')    
    for i in temp:
        link_list.append('https://market.cetizen.com'+i['href'])

100%|██████████████████████████████████████████████████████████████████████████████████| 93/93 [00:31<00:00,  2.96it/s]


In [66]:
len(link_list)

2790

## 데이터프레임으로 

In [87]:
data=[]

for link in tqdm(link_list):
    url=link
    req=requests.get(url,'html.parser')
    soup=BeautifulSoup(req.text)

    if len(soup.find_all('div','box_good'))==5:
        temp=[]
        # 이름
        try:
            temp.append(soup.select('#main_container > div:nth-child(13) > div.col-4 > div > div:nth-child(1) > div.col.black1 > div > div > span.fb.fsize7' )[0].text )
        except:
            temp.append('이름없음')
        
        # 속성들
        for i in range(5):
            try:
                temp.append(soup.find_all('div','box_good')[i].text.strip())
            except:
                temp.append('')
        
        
        # 가격
        try:
            temp.append(soup.find_all('span','text-right fsize11 red pl-2')[0].text)
        except:
            temp.append('가격없음')

        data.append(temp)

100%|██████████████████████████████████████████████████████████████████████████████| 2790/2790 [14:10<00:00,  3.28it/s]


## 가격 전처리

In [128]:
# 가격 없는 애들 None값 처리
for i in range(len(data)):
    if test.loc[i,'가격']=='가격없음':
        test.loc[i,'가격']=None

In [134]:
# 가격 int로 변경
test['가격']=test['가격'].astype(int)

In [173]:
test.groupby('보증기간')['가격'].median()

보증기간
기간종료    380000.0
미확인     480000.0
보증가능    750000.0
Name: 가격, dtype: float64

+ 제품상태/구성품은 가격에 영향 크게 미침
+ 기변 상태는 별로
+ 선택약정 -> 25%할인/약정불가 가격차이 좀 있음
+ 보증기간 굉장히 영향이 큼!!

## 기기명 -> 시리즈/모델/용량 전처리

In [146]:
test['용량']=test['기기명'].str.split(' ').str[-1]

C:\Users\user\AppData\Local\Temp\ipykernel_1028\553252274.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['용량']=test['기기명'].str.split(' ').str[-1]


In [152]:
test['분류1']=test['기기명'].str.extract(r'(아이폰.{0,1}[0-9]+)') #시리즈

In [189]:
test['분류2']=test['기기명'].str.extract(r'아이폰[0-9]+(.*[가-힣]+)') #모델